In [1]:
# Import necessary libraries
import h5py
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from scipy.optimize import minimize


# Task 1
## Subtask 1

In [2]:
hf = h5py.File('regression.h5', 'r')
x_train = np.array(hf.get('x_train'))
y_train = np.array(hf.get('y_train'))
x_test = np.array(hf.get('x_test'))
y_test = np.array(hf.get('y_test'))
hf.close()


In [3]:
class RegularizedKernelizedLogisticRegression:
    """
    Regularized Kernelized Logistic Regression.

    Parameters:
    - kernel_function: Callable
        A kernel function that computes the kernel matrix for the input data.
    - alpha: float, optional (default=1e-5)
        Regularization parameter controlling the strength of regularization.
    - max_iter: int, optional (default=1000)
        Maximum number of iterations for the optimization algorithm.
    - random_state: int, optional (default=42)
        Seed for reproducibility of random initialization.

    Attributes:
    - kernel_function: Callable
        The provided kernel function.
    - alpha: float
        Regularization parameter.
    - max_iter: int
        Maximum number of iterations for optimization.
    - random_state: int
        Seed for random initialization.
    - w: numpy.ndarray
        Optimized weights after fitting the model.
    """

    def __init__(self, kernel_function, alpha=1e-5, max_iter=1000, random_state=42):
        """
        Initialize the RegularizedKernelizedLogisticRegression instance.

        Parameters:
        - kernel_function: Callable
            A kernel function that computes the kernel matrix for the input data.
        - alpha: float, optional (default=1e-5)
            Regularization parameter controlling the strength of regularization.
        - max_iter: int, optional (default=1000)
            Maximum number of iterations for the optimization algorithm.
        - random_state: int, optional (default=42)
            Seed for reproducibility of random initialization.
        """
        self.kernel_function = kernel_function
        self.alpha = alpha
        self.max_iter = max_iter
        self.random_state = random_state
        self.w = None

    def fit(self, x, y):
        """
        Fit the model to the training data.

        Parameters:
        - x: numpy.ndarray
            Training data.
        - y: numpy.ndarray
            Target values.

        Returns:
        None
        """
        n_samples, _ = x.shape

        # Precompute kernel matrix
        K = self.kernel_function(x)

        # Initialize weights
        np.random.seed(self.random_state)
        self.w = np.random.rand(n_samples)

        # Define logistic loss and its gradient
        def logistic_loss(w):
            logits = K @ w
            logits = logits.clip(-500, 500)
            predictions = 1 / (1 + np.exp(-logits))
            epsilon = 1e-5
            loss = -np.sum(y * np.log(predictions + epsilon) + (1 - y) * np.log(1 - predictions + epsilon)) / n_samples
            regularization_term = self.alpha * w @ (K @ w)
            return loss + regularization_term

        def logistic_gradient(w):
            logits = K @ w
            logits = logits.clip(-500, 500)
            predictions = 1 / (1 + np.exp(-logits))
            error = predictions - y
            gradient = (K.T @ error) / n_samples + 2 * self.alpha * (K @ w)
            return gradient

        # Minimize the logistic loss
        result = minimize(logistic_loss, self.w, jac=logistic_gradient, method='L-BFGS-B', options={'maxiter': self.max_iter})

        # Store the optimized weights
        self.w = result.x

    def predict(self, x):
        """
        Predict class labels for new data.

        Parameters:
        - x: numpy.ndarray
            New data for prediction.

        Returns:
        numpy.ndarray
            Predicted class labels (binary).
        """
        # Compute kernel matrix for new data
        K_new = self.kernel_function(x)

        # Compute logits and predict probabilities
        logits = K_new @ self.w
        logits = logits.clip(-500, 500)
        probabilities = 1 / (1 + np.exp(-logits))

        # Convert probabilities to binary predictions
        predictions = (probabilities >= 0.5).astype(int)

        return predictions


In [4]:
def rbf(x):
    """
    Calculate the Radial Basis Function (RBF) kernel matrix.

    Parameters:
    - x (numpy.ndarray): Input data, where each row represents a data point.

    Returns:
    numpy.ndarray: RBF kernel matrix.
    """
    # Calculate pairwise squared Euclidean distances
    dists = np.square(np.linalg.norm(x[:, np.newaxis] - x, axis=2))
    
    # Compute the RBF kernel matrix
    K = np.exp(-0.5 * dists)
    
    return K


In [5]:
scaler = StandardScaler()

# Fit the scaler on your data and transform it
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.fit_transform(x_test)

# for the first subtask we can only use the first 2 features
x_train_scaled_geo = x_train_scaled[:, :2]
x_test_scaled_geo = x_test_scaled[:, :2]

# splitting the two target variables separately
y_train_chimney = y_train[:, 0]
y_test_chimney = y_test[:, 0]
y_test_children = y_test[:, 1]


In [9]:
# the alphas to consider
alphas = [10, 5, 1, 0.1, 1e-3, 1e-5]

best_model = None
best_accuracy = 0

# chossing the best alpha wrt. the validation set accuracy (hyperparameter tuning)
for alpha in alphas:
    
    model = RegularizedKernelizedLogisticRegression(rbf, alpha)
    model.fit(x_train_scaled_geo, y_train_chimney)
    y_test_pred = model.predict(x_test_scaled_geo)
    accuracy_test = accuracy_score(y_test_chimney, y_test_pred)
    # if the new model has a better accuracy than previous data, update it
    if accuracy_test > best_accuracy:
        best_accuracy = accuracy_test
        best_model = model


In [10]:
# reporting the classification accuracy on the training and validation data
y_train_pred = best_model.predict(x_train_scaled_geo)
training_accuracy = accuracy_score(y_train_chimney, y_train_pred)

print(f'The classification accuracy on the training data: {training_accuracy*100}%')
print(f'The classification accuracy on the validation data: {best_accuracy*100}%')


The classification accuracy on the training data: 64.9%
The classification accuracy on the validation data: 58.099999999999994%


## Subtask 2

In [8]:
def custom_kernel(x):
    """
    Computes a custom kernel function as a combination of three RBF kernels.

    Parameters:
    - x: Input data matrix.

    Returns:
    - The result of the custom kernel computation.
    """
    return rbf(x)**3 + rbf(x)**2 + rbf(x)


As we knew that a polynomial with positive coefficients of a kernel is a kernel, I tried out some polynomial variations of the `rbf` kernel, and with the `rbf^3 + rbf^2 + rbf` kernel and with using all the available features I was able to achieve better accuracy on the test data.

In [9]:
best_improved_model = None
best_accuracy = 0

# chossing the best alpha wrt. the validation set accuracy (hyperparameter tuning)
# we use the same alpha pool as before
for alpha in alphas:
    
    improved_model = RegularizedKernelizedLogisticRegression(custom_kernel, alpha)
    improved_model.fit(x_train_scaled, y_train_chimney)
    y_test_pred = improved_model.predict(x_test_scaled)
    accuracy_test_improved = accuracy_score(y_test_chimney, y_test_pred)
    # if the new model has a better accuracy than previous data, update it
    if accuracy_test_improved > best_accuracy:
        best_accuracy = accuracy_test_improved
        best_improved_model = improved_model


In [10]:
# reporting the classification accuracy on the training and validation data and the chosenalpha hyperparameter
y_train_pred = best_improved_model.predict(x_train_scaled)
training_accuracy = accuracy_score(y_train_chimney, y_train_pred)

print(f'The classification accuracy on the training data: {training_accuracy*100}%')
print(f'The classification accuracy on the validation data: {best_accuracy*100}%')
print(f'The selected alpha hyperparameter: {best_improved_model.alpha}')


The classification accuracy on the training data: 79.10000000000001%
The classification accuracy on the validation data: 64.3%
The selected alpha hyperparameter: 1


## Subtask 3
Similarly to the original cost function, our cost function will be:

`c(y, y^) = y⋅(1−y^)−(1−y)⋅(1−y^)-10⋅(1−y)⋅(y^) = y⋅(1−y^)−(1−y)⋅(9⋅(y^) + 1)`

where:
- y is the true label,
- y^ is the predicted probability

as in this case for the inputs given in the description the ouput is always the required value, because in this case `y` denotes if Santa really visits the given house.

In [11]:
def cost_support(a, y):
    """
    Calculates the custom logistic regression cost function with support for top predictions.

    Parameters:
    - a: Predicted probabilities.
    - y: True labels.

    Returns:
    - The custom logistic regression cost.
    """
    n_samples = len(y)
    cost = np.sum(y * (1 - a) + (1 - y) * (9 * a + 1)) / n_samples
    
    return cost

class RegularizedKernelizedLogisticRegressionSupport:
    def __init__(self, kernel_function, alpha=1e-5, max_iter=1000, random_state=42, limit=50):
        """
        Regularized Kernelized Logistic Regression model with support for top predictions.

        Parameters:
        - kernel_function: Callable
            A kernel function that computes the kernel matrix for the input data.
        - alpha: float, optional (default=1e-5)
            Regularization parameter controlling the strength of regularization.
        - max_iter: int, optional (default=1000)
            Maximum number of iterations for the optimization algorithm.
        - random_state: int, optional (default=42)
            Seed for reproducibility of random initialization.
        - limit: int, optional (default is 50)
            Number of top predictions to consider
        

        Attributes:
        - kernel_function: Callable
            The provided kernel function.
        - alpha: float
            Regularization parameter.
        - max_iter: int
            Maximum number of iterations for optimization.
        - random_state: int
            Seed for random initialization.
        - w: numpy.ndarray
            Optimized weights after fitting the model.
        """
        self.kernel_function = kernel_function
        self.alpha = alpha  # regularization parameter
        self.max_iter = max_iter
        self.random_state = random_state
        self.limit = limit
        self.w = None

    def fit(self, x, y):
        """
        Fit the model to the training data.

        Parameters:
        - x: numpy.ndarray
            Training data.
        - y: numpy.ndarray
            Target values.

        Returns:
        None
        """
        n_samples, _ = x.shape

        # Precompute kernel matrix
        K = self.kernel_function(x)

        # Initialize weights
        np.random.seed(self.random_state)
        self.w = np.random.rand(n_samples)

        # Define logistic loss and its gradient
        def logistic_loss(w):
            logits = K @ w
            logits = logits.clip(-500, 500)
            probabilities = 1 / (1 + np.exp(-logits))
            # Find the indices of the 50 largest values
            top_indices = np.argsort(probabilities)[-self.limit:]

            # Create a new array of zeros
            predictions = np.zeros_like(probabilities)

            # Set the elements at the identified indices to 1
            predictions[top_indices] = 1

            loss = cost_support(y, predictions)
            regularization_term = self.alpha * w @ (K @ w)
            return loss + regularization_term

        def logistic_gradient(w):
            logits = K @ w
            logits = logits.clip(-500, 500)
            predictions = 1 / (1 + np.exp(-logits))
            error = predictions - y
            gradient = (K.T @ error) / n_samples + 2 * self.alpha * (K @ w)
            return gradient

        # Minimize the logistic loss
        result = minimize(logistic_loss, self.w, jac=logistic_gradient, method='L-BFGS-B', options={'maxiter': self.max_iter})

        # Store the optimized weights
        self.w = result.x

    def predict(self, x):
        """
        Predict class labels for new data.

        Parameters:
        - x: numpy.ndarray
            New data for prediction.

        Returns:
        numpy.ndarray
            Predicted class labels (binary).
        """
        # Compute kernel matrix for new data
        K_new = self.kernel_function(x)

        # Compute logits and predict probabilities
        logits = K_new @ self.w
        logits = logits.clip(-500, 500)
        probabilities = 1 / (1 + np.exp(-logits))

        # Find the indices of the 50 largest values
        top_indices = np.argsort(probabilities)[-self.limit:]

        # Create a new array of zeros
        predictions = np.zeros_like(probabilities)

        # Set the elements at the identified indices to 1
        predictions[top_indices] = 1

        return predictions


In [12]:
# the alphas to consider during hyperparameter tuning
alphas = [5000, 1000, 100, 10, 5, 1, 0.1, 1e-3, 1e-5]

best_model = None
best_cost = np.inf

# chossing the best alpha wrt. the cost of the validation set (hyperparameter tuning)
for alpha in alphas:
    
    model = RegularizedKernelizedLogisticRegressionSupport(custom_kernel, alpha)
    model.fit(x_train_scaled, y_train_chimney)
    y_test_pred = model.predict(x_test_scaled)
    cost = cost_support(y_test_pred, y_test_chimney)
    # if the new model has a better accuracy than previous data, update it
    if cost < best_cost:
        best_cost = cost
        best_model = model

print(f'The cost achieved: {best_cost}')


The cost achieved: 1.25


In [13]:
# calculating the cost on random chossing 50 houses for Santa to visit
num_random_selections = 50
limit = 50
num_of_samples = len(y_test_chimney)

cost_random = []

for i in range(num_random_selections):
    np.random.seed(i)

    # Create an array of length 50 with all zeros
    y_test_pred_random = np.zeros(num_of_samples)

    # Choose 10 random indices to set to 1
    random_indices = np.random.choice(num_of_samples, limit, replace=False)

    # Set the chosen indices to 1
    y_test_pred_random[random_indices] = 1
    
    cost = cost_support(y_test_pred_random, y_test_chimney)
    cost_random.append(cost)

cost_random = sum(cost_random) / len(cost_random)
# reporting the average cost of 50 random samples
print(f'The cost achieved in a random way: {cost_random}')


The cost achieved in a random way: 1.2706000000000004


Meaning that my model has only a slightly better performance than picking the houses randomly.

In [14]:
# reporting the houses with children Santa visited
visited_houses_with_children = np.sum(np.logical_and(y_test_children, y_test_pred).astype(int))
print(f'Santa visits {visited_houses_with_children} houses with children')


Santa visits 30 houses with children
